<a href="https://colab.research.google.com/github/noircir/Python/blob/master/07_LDA_Latent_Dirichlet_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latent Dirichlet Allocation

Add more documents: each document is a line of text in a pandas dataframe.

In [0]:
!pip install docx2txt

In [0]:
import pandas as pd
import docx2txt
import re

## Read Word documents, convert them to text format, compress a bit (remove blank lines), and create a dataframe with a document per row. 

In [0]:
def compress(text):
  '''
  removes blank lines and replaces multiple spaces with one space
  '''
  text = text.replace('\t', ' ')
  return re.sub('\n+', '\n', text)

In [0]:
from itertools import dropwhile

def remove_TOC(text):

  # drop list items until find a string with a substring "ANNEXE"
  temp = list(dropwhile(lambda sentence: "ANNEXE" not in sentence, text))

  # In the first line, drop everything until the word 'CONTRAT'
  index = temp[0].find('CONTRAT')
  temp[0] = temp[0][index:]
  return temp

In [0]:
docs = ['document6.docx', 'document8.docx', 'document10.docx', 'document11.docx', 
          'document14.docx', 'document15.docx', 'document16.docx', 'document17.docx', 
          'document18.docx']
documents = []
for doc in docs:
  temp = docx2txt.process('/content/drive/My Drive/Colab Notebooks/Self-learning chatbot/texts/' + doc)
  #temp = remove_TOC(text)
  temp = compress(temp)
  documents.append(temp)

In [0]:
d = {'text': documents}
df = pd.DataFrame(data=d)

In [0]:
df.head(10)

,text
0,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
1,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
2,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
3,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
4,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
5,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
6,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
7,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
8,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...


## Preprocessing

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [0]:
stopwords = list({'ne', 'j', 'aie', 'eusse', 'l', 's', 'étés', 'la', 'étais', 'soient', 'furent', 'eûtes', 'avez', 'nous', 'leur', 'ait', 'avions', 'fusses', 'eurent', 'suis', 'aurions', 'te', 'étante', 'eu', 'aurez', 'le', 'ce', 'nos', 'fussent', 'avais', 'sera', 'eussent', 'ayants', 'et', 'auraient', 'en', 'au', 'une', 'ma', 'pas', 'étées', 'fussions', 'du', 'était', 'étaient', 'il', 'vous', 'avaient', 'tes', 'd', 'n', 'fusse', 'êtes', 'eut', 'eût', 'étiez', 'seras', 'eue', 'sont', 'serions', 'aurais', 'aux', 'étant', 'eus', 'étantes', 'seriez', 'des', 'auront', 'sommes', 'ces', 't', 'qu', 'fut', 'soyons', 'fût', 'ton', 'lui', 'ses', 'aurai', 'ayez', 'eux', 'ont', 'eussions', 'dans', 'seront', 'serais', 'seraient', 'moi', 'eussiez', 'fûmes', 'aura', 'soit', 'soyez', 'ai', 'fussiez', 'ta', 'm', 'à', 'eues', 'mais', 'aurait', 'par', 'un', 'serait', 'les', 'vos', 'eûmes', 'qui', 'je', 'même', 'étions', 'de', 'as', 'notre', 'aviez', 'ils', 'est', 'sur', 'votre', 'étants', 'avons', 'ayante', 'on', 'aurons', 'avait', 'eusses', 'ayantes', 'son', 'sa', 'es', 'serons', 'serez', 'toi', 'auriez', 'aies', 'étée', 'se', 'ayant', 'me', 'tu', 'mon', 'ayons', 'avec', 'fus', 'pour', 'ou', 'y', 'que', 'fûtes', 'auras', 'mes', 'aient', 'été', 'sois', 'elle', 'serai', 'c'})

In [0]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words= stopwords)

In [0]:
# Document-term metrics

dtm = cv.fit_transform(df['text'])

In [0]:
dtm

<9x1516 sparse matrix of type '<class 'numpy.int64'>'
	with 7012 stored elements in Compressed Sparse Row format>

# LDA

In [0]:
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=5,random_state=42)

LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

## Showing Stored Words

In [0]:
len(cv.get_feature_names())

1516

In [0]:
import random

for i in range(10):
    random_word_id = random.randint(0,1515)
    print(cv.get_feature_names()[random_word_id])

continue
etc
excède
antérieur
bien
identifiées
faits
attribuées
dédommager
exécutable


In [0]:
for i in range(10):
    random_word_id = random.randint(0,1515)
    print(cv.get_feature_names()[random_word_id])

exclusivement
intentées
réfléchi
facturer
présence
maintient
décidé
établies
fiches
contenant


## Showing Top Words per Topic

In [0]:
len(LDA.components_)

5

In [0]:
type(LDA.components_)

numpy.ndarray

In [0]:
LDA.components_.shape

(5, 1516)

In [0]:
# numpy arrays containing probabilites of the words
LDA.components_

array([[ 0.20000017,  4.19999972,  0.20000023, ...,  0.20000015,
         0.20000009,  0.20000018],
       [ 3.14407284,  0.20000001,  1.45284342, ..., 10.06986777,
         0.20000002,  2.33051921],
       [ 0.20000066,  0.20000024,  0.20000087, ...,  0.20000058,
         0.20000034,  0.20000068],
       [ 3.25590815,  0.20000001,  3.91911051, ..., 13.33005225,
         2.19999952,  6.03483018],
       [ 3.20001818,  0.20000002,  1.22804496, ...,  0.20007925,
         0.20000003,  2.23464975]])

In [0]:
len(LDA.components_[0])

1516

In [0]:
single_topic = LDA.components_[0]

In [0]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([1133,  581, 1029, ...,  981,  870, 1008])

In [0]:
# Word least representative of this topic
single_topic[1008]

28.199860683383356

In [0]:
# Word most representative of this topic
single_topic[1133]

0.20000004421365866

In [0]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([  81,  871, 1475, 1477,   48, 1058, 1003,  981,  870, 1008])

In [0]:
top_word_indices = single_topic.argsort()[-10:]

In [0]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

addenda
matériel
échéance
échéancier
____________________________________
professionnel
phase
ouvrage
matériaux
plans


It is unclear what topic this is... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 5 topics found.

In [0]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['certificat', 'juridiquement', 'organisé', 'tard', 'actif', 'addenda', 'matériel', 'échéance', 'échéancier', '____________________________________', 'professionnel', 'phase', 'ouvrage', 'matériaux', 'plans']


THE TOP 15 WORDS FOR TOPIC #1
['période', 'pre', 'numéro', 'matériel', 'auteur', 'pourcentage', 'exigences', 'évaluation', 're', 'rendement', 'engagement', 'cession', 'licence', 'destruction', 'logiciel']


THE TOP 15 WORDS FOR TOPIC #2
['confirmer', 'prévoit', 'efficace', 'civil', 'nommée', 'fournissant', 'suivi', 'domaine', 'verbal', 'éliminant', 'transmettant', 'obstacle', 'empêchant', 'désigné', 'matériaux']


THE TOP 15 WORDS FOR TOPIC #3
['fournisseurs', 'nouveau', 'bon', 'période', 'rendement', 'initiale', 'numéro', 'option', 'exigences', 'pourcentage', 'commande', 'destruction', 'consignation', 'ouvrage', 'bien']


THE TOP 15 WORDS FOR TOPIC #4
['guise', 'conformes', '65', 'admissibles', 'convient', 'horaire', 'tarif', 'facture', 'facturatio

## Attaching Discovered Topic Labels to Original Articles

In [0]:
dtm

<9x1516 sparse matrix of type '<class 'numpy.int64'>'
	with 7012 stored elements in Compressed Sparse Row format>

In [0]:
dtm.shape

(9, 1516)

In [0]:
len(df)

9

In [0]:
topic_results = LDA.transform(dtm)

In [0]:
topic_results.shape

(9, 5)

In [0]:
# This is the probability of document belnging to a particular topic 
# (before, it was probabilities of words belonging to a particular topic)

topic_results[0]

array([9.98240988e-05, 1.01497938e-04, 9.91649861e-05, 9.99598168e-01,
       1.01345442e-04])

In [0]:
topic_results[0].round(2)

array([0., 0., 0., 1., 0.])

In [0]:
topic_results[0].argmax()

3

This means that our model thinks that the first article belongs to topic #3.

## Combining with Original Data

In [0]:
df.head()

,text
0,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
1,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
2,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
3,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...
4,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...


In [0]:
topic_results.argmax(axis=1)

array([3, 3, 1, 4, 0, 0, 3, 3, 1])

In [0]:
# add a column of topics
df['Topic'] = topic_results.argmax(axis=1)

In [0]:
df.head(10)

,text,Topic
0,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...,3
1,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...,3
2,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...,1
3,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...,4
4,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...,0
5,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...,0
6,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...,3
7,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...,3
8,APPEL D’OFFRES NO\n DOCUMENT D’APPEL D’OFFRES ...,1


docs = ['document6.docx' -- Biens (fournitures),

'document8.docx', -- Haute-technologie

'document10.docx', -- Service professionnels

'document11.docx', -- Service professionnels lies a la construction

'document14.docx', -- Travaux de construction et services professionnels

'document15.docx',-- Travaux de construction et services professionnels 2 etapes

'document16.docx', -- Biens informatiques

'document17.docx', -- Biens informatiques - equipement

'document18.docx'] -- Services informatiques